In [4]:
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
from tqdm import tqdm

# Custom libraries
import utils
from diffusion import DiffusionModel

In [13]:
#! Settings
#DEVICE = "cuda:0"
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
LR = 0.001
NO_EPOCHS = 10000
BATCH_SIZE = 5
PRINT_FREQUENCY = 100
VERBOSE = True
MARGIN = 0.004
IMAGE_SHAPE = (64, 64)

SAVE_MODEL_DIR = "/Volumes/ML/Simple-Diffusion-Model-From-Scratch/"

In [7]:
print(torch.backends.mps.is_available())  # Should return True if MPS is supported


True


In [10]:
#! Prepare data
img0 = Image.open("data/players/beckham.jpg").convert('RGB')
img1 = Image.open("data/players/messi.jpg").convert('RGB')
img2 = Image.open("data/players/ronaldo.png").convert('RGB')
img3 = Image.open("data/players/haaland.jpg").convert('RGB')
img4 = Image.open("data/players/silva.jpg").convert('RGB')


img0_tensor = utils.transform(img0)
img1_tensor = utils.transform(img1)
img2_tensor = utils.transform(img2)
img3_tensor = utils.transform(img3)
img4_tensor = utils.transform(img4)


batch = torch.stack([img0_tensor, img1_tensor, img2_tensor, img3_tensor, img4_tensor]).to(DEVICE)
label = torch.tensor([0,1,2,3,4]).reshape(-1,1).float().to(DEVICE)

print("Input Batch Images:", batch.shape, batch.dtype, batch.device)
print("Input Batch Labels:", label.shape, label.dtype, label.device)

Input Batch Images: torch.Size([5, 3, 64, 64]) torch.float32 mps:0
Input Batch Labels: torch.Size([5, 1]) torch.float32 mps:0


In [11]:
class UNet(nn.Module):
    def __init__(self):
      super().__init__()
      # Complete code here
      channels = (64, 128, 256, 512, 1024)
      down_channels = list(reversed(channels))
      time_emb_dim = 32
      label = False

      self.conv1 = nn.Conv2d(3, channels[0], 3, padding=1)
      self.relu = nn.ReLU()

      #DownScale
      self.downscales = nn.ModuleList([utils.Block(channels[i], channels[i+1], 100, label) for i in range(len(channels)-1)])

      #UpScale
      self.upscales = nn.ModuleList([utils.Block(down_channels[i], down_channels[i+1], 100, label, downsample=False) for i in range(len(channels)-1)])

      #output
      self.output = nn.Conv2d(64, 3, 1)


    def forward(self, x, t, **kwargs):
        # Complete code here
        x = self.relu(self.conv1(x))
        # Unet
        residual_inputs = []
        for downscale in self.downscales:
            x = downscale(x, t)
            residual_inputs.append(x)
        for upscale in self.upscales:
            residual_x = residual_inputs.pop()
            # Add residual x as additional channels
            x = torch.cat((x, residual_x), dim=1)
            x = upscale(x, t)
        return self.output(x)

model = UNet()
print("Num params: ", sum(p.numel() for p in model.parameters()))
model

Num params:  62633667


UNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (downscales): ModuleList(
    (0): Block(
      (time_embedding): SinusoidalPositionEmbeddings()
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (final): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (bnorm1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (time_mlp): Linear(in_features=100, out_features=128, bias=True)
      (relu): ReLU()
    )
    (1): Block(
      (time_embedding): SinusoidalPositionEmbeddings()
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (final): Conv2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (bnorm1): BatchNorm

In [12]:
#! Define Diffusion Model
diffusion_model = DiffusionModel()

#! Define Unet
unet = UNet().to(DEVICE)

#! Define Optimizer
optimizer = torch.optim.Adam(unet.parameters(), lr=LR)

In [14]:
#! Training
MODEL_DIR = f"{SAVE_MODEL_DIR}/trained-model.pt"
for epoch in tqdm(range(NO_EPOCHS)):
    # 1. Get B random integers between (0, timestep)
    # Hint use: torch.randint()
    t = torch.randint(0, 300, (BATCH_SIZE,)).to(DEVICE)

    # 2. Forward diffuse each image with different timestep values
    # Hint use: diffusion_model.forward() function
    img_plus_noise, noise = diffusion_model.forward(batch, t, DEVICE)
    # 3. Predict the noise in this batch using Unet
    predicted_noise = unet(img_plus_noise, t, labels=label)

    # 4. Calculate the MSE loss between actual noise and predicted noise
    # Hint use: torch.nn.functional.mse_loss
    mse_loss = nn.functional.mse_loss(noise, predicted_noise)
    # 5. Make all gradients zero
    optimizer.zero_grad()

    # 6. Calculate Gradients
    mse_loss.backward()

    # 7. Update the Model
    optimizer.step()

    # 8. Print the Loss on the intervel of given PRINT_FREQUENCY
    if epoch % PRINT_FREQUENCY == 0:
      print(f"Current epoch: {epoch} ------ loss: {mse_loss} ----------")


    # 9. if loss < MARGIN, save the model and exit the training process
    # Hint use torch.save() and model.state_dict()
    if mse_loss < MARGIN:
      print(f"loss less than margin---------- {mse_loss} ------ {MARGIN}")
      torch.save(unet.state_dict(), MODEL_DIR)
      break

# 10. save the model once more at the end
torch.save(unet, MODEL_DIR)

  0%|                                                                               | 1/10000 [00:07<19:52:02,  7.15s/it]

Current epoch: 0 ------ loss: 1.0241875648498535 ----------


  1%|▊                                                                               | 102/10000 [00:19<19:38,  8.40it/s]

Current epoch: 100 ------ loss: 0.1168253943324089 ----------


  2%|█▌                                                                              | 202/10000 [00:31<19:26,  8.40it/s]

...

 74%|██████████████████████████████████████████████████████████▍                    | 7402/10000 [14:58<05:11,  8.35it/s]

Current epoch: 7400 ------ loss: 0.013154572807252407 ----------


 75%|██████████████████████████████████████████████████████████▉                    | 7466/10000 [15:06<05:02,  8.37it/s]

loss less than margin---------- 0.0038284542970359325 ------ 0.004


 75%|██████████████████████████████████████████████████████████▉                    | 7466/10000 [15:07<05:07,  8.23it/s]


In [15]:
#! Reverse DIffusion
LABEL = 1

In [17]:
with torch.no_grad():
    # 1. Generate random noise
    img = torch.randn((1, 3) + IMAGE_SHAPE).to(DEVICE)
    for i in tqdm(reversed(range(diffusion_model.timesteps))):

        t = torch.full((1,), i, dtype=torch.long, device=DEVICE)
        l = torch.tensor(LABEL, dtype=torch.float, device=DEVICE).reshape(-1,1)

        # Reverse diffusion
        img = diffusion_model.backward(img, t, unet.eval(), labels = l)

        img_np = utils.reverse_transform(img[0])
        img_np = img_np.resize((256, 256))
        img_np.save(f"{SAVE_MODEL_DIR}/output-img/L{LABEL}-{i}.png")

300it [00:05, 51.31it/s]


In [18]:
diffusion_model.timesteps

300